In [31]:
# Setting the root directory
%cd /Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt

/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt


In [32]:
# Variable definations
path = '/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/'

CATEGORIES = ['MGMT_negative', 'MGMT_positive']

block_h, block_w = (224, 224)
# stride = 3

Case_Num = 5

# Modality to target
modality = 'flair'

# define dataframes globally
import pandas as pd
df_patient_final_mgmtpos = pd.DataFrame()
df_patient_final_mgmtneg = pd.DataFrame()
global df_patient_final_mgmtneg, df_patient_final_mgmtpos


Creating two types of Dataframes for storing the features:
1. df_patients:
Procedure : 
Will go thorough all the patients individually and pass it through the model to get the features from fc1 and fc2 layers. It will then concat the two variables coloumn wise and store it in a dataframe.
This will contain all the features from a single patient and then it will take an avarage of all the features and store it in the final dataframe.
2. df_patients_final:
Procedure :
After we have the invidual avarage features from all the patients we would then label the


In [33]:
# Importing necceary model architecture
from keras.applications.vgg19 import VGG19  # VGG19 model
from keras.models import Model
from keras.optimizers import SGD
from keras.applications.resnet import ResNet50

# Loading the VGG19 model
base_model_VGG19 = VGG19(include_top=True, weights='imagenet',
                   input_shape=(block_h, block_w, 3))

# Loading the AlexNet Model

# Loading the ResNet Model
base_model_ResNet = ResNet50(include_top=True, weights='imagenet', input_shape=(block_h, block_w,3))

# getting fc1 and fc2 layers
model_fc1 = Model(inputs=base_model_ResNet.input,
                  outputs=base_model_ResNet.get_layer('predictions').output)
# model_fc2 = Model(inputs=base_model_ResNet.input,
#                   outputs=base_model_ResNet.get_layer('fc2').output)
 

In [34]:
base_model_ResNet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_11[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [35]:
# Passing the list of images through the VGG19 model
import os
import cv2
import numpy as np
from keras_preprocessing.image import load_img
import pandas as pd
from keras.applications.resnet import preprocess_input

def patient_image_vgg19(Patient_Path, category):
    os.chdir(Patient_Path)
    patientpath= os.listdir(Patient_Path)
    if '.DS_Store' in patientpath:
        patientpath.remove('.DS_Store')
    # print(patientpath)
    # datafrane declarations for storing the features for respective categories
    df_patient_mgmtpos = pd.DataFrame()
    df_patient_mgmtneg = pd.DataFrame()
    for image in patientpath:
        # imread cv2
        img = cv2.imread(image)
        # print(img.shape)
        img = cv2.resize(img, (block_h, block_w))
        # print(img.shape)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)
        # pass it throught the model
        fc1_features = model_fc1.predict(img, verbose=0)
        # fc2_features = model_fc2.predict(img, verbose=0)
        df_fc1_features = pd.DataFrame(fc1_features)
        # df_fc2_features = pd.DataFrame(fc2_features)
        # Function to rename the column of fc2 dataframe for easier calculation
        # dict = {}
        # for i in range(4096):
        #     dict.update({i:i+4096})
        # # print(dict)
        # df_fc2_features = df_fc2_features.rename(columns=dict)
        # print(df_fc2_features.head)
        # concat the features to the dataframe for patient
        # df_patient = pd.concat([df_fc1_features,df_fc2_features], axis=1)
        df_patient = df_fc1_features
        if category == 'MGMT_positive':
            df_patient_mgmtpos = pd.concat([df_patient_mgmtpos,df_patient], axis=0)
        else:
            df_patient_mgmtneg = pd.concat([df_patient_mgmtneg,df_patient], axis=0)
    return df_patient_mgmtpos, df_patient_mgmtneg  # returning to Patient_Data() function
        

In [36]:
# Reading the data and creating the required Dataframe
import os
import pandas as pd
def Patient_Data(Split_Path,case, df_patient_final_mgmtpos,df_patient_final_mgmtneg):
    from tqdm import tqdm
    import cv2
    # datafrane declarations for storing the features for respective categories
    df_patient_mgmtpos = pd.DataFrame()
    df_patient_mgmtneg = pd.DataFrame()
    # df_patient_final_mgmtpos = pd.DataFrame()
    # df_patient_final_mgmtneg = pd.DataFrame()
    print("Reading data for {} : ".format(case))
    splitpath= os.listdir(Split_Path)
    if '.DS_Store' in splitpath:
        splitpath.remove('.DS_Store')
    for category in splitpath:
        Category_Path = os.path.join(Split_Path,category)
        categorypath = os.listdir(Category_Path)
        if '.DS_Store' in categorypath:
            categorypath.remove('.DS_Store')
        for patient in tqdm(categorypath):
            Patient_Path = os.path.join(Category_Path,patient)
            # here we have reached individual patient folder and we need to read the data as well as pass it through the VGG19 model
            # print("Reading data for {} : ".format(patient))
            df_patient_mgmtpos, df_patient_mgmtneg = patient_image_vgg19(Patient_Path, category)
            # avaraging the values for each patient axis = 0
            df_patient_mgmtpos = df_patient_mgmtpos.mean(axis=0)
            df_patient_mgmtneg = df_patient_mgmtneg.mean(axis=0)
            # print(df_patient_mgmtpos.shape)
            # print(df_patient_mgmtneg.shape)
            # os.chdir(Category_Path)
            # if category == 'MGMT_positive':
            #     df_patient_mgmtpos.to_csv('MGMT_POS-{}.csv'.format(patient))
            # else:
            #     df_patient_mgmtneg.to_csv('MGMT_NEG-{}.csv'.format(patient))
            # concating the patient features to the final dataframe axis = 0
            if category == 'MGMT_positive':
                df_patient_final_mgmtpos = pd.concat([df_patient_final_mgmtpos,df_patient_mgmtpos], axis=1)
            else:
                df_patient_final_mgmtneg = pd.concat([df_patient_final_mgmtneg,df_patient_mgmtneg], axis=1)
            del df_patient_mgmtpos, df_patient_mgmtneg
        # print(df_patient_final_mgmtneg.describe())
        # print(df_patient_final_mgmtpos.describe())
        os.chdir(Category_Path)
        # transpose the dataframe
        # df_patient_final_mgmtneg = df_patient_final_mgmtneg.transpose()
        # df_patient_final_mgmtpos = df_patient_final_mgmtpos.transpose()
        # if category == 'MGMT_positive':
        #     df_patient_final_mgmtpos.to_csv('MGMT_POS-{}.csv'.format(case))
        # else:
        #     df_patient_final_mgmtneg.to_csv('MGMT_NEG-{}.csv'.format(case))
    return df_patient_final_mgmtpos.transpose(), df_patient_final_mgmtneg.transpose() # returning to main cell named All Function Calls

In [37]:
# Removing columns having MAD less than 0.5
import pandas as pd

def MAD(df_patient_mgmtpos, df_patient_mgmtneg,case):
    print("Eleminating columns by MAD : \n")
    df_patient_mgmtpos_mad = df_patient_mgmtpos.mad(axis=0)
    df_patient_mgmtneg_mad = df_patient_mgmtneg.mad(axis=0)
    # print(len(df_patient_mgmtneg_mad), len(df_patient_mgmtpos_mad))
    # df_patient_mgmtpos_mad.to_csv('MGMT-MAD-POS-{}.csv'.format(case))
    # df_patient_mgmtneg_mad.to_csv('MGMT-MAD-NEG-{}.csv'.format(case))
    mgmt_column =[]
    for i in range(len(df_patient_mgmtpos_mad)):
        # print(df_patient_mgmtneg_mad[i])
        if df_patient_mgmtpos_mad[i] < 0.5:
            if df_patient_mgmtneg_mad[i] < 0.5:
                mgmt_column.append(i)
    df_patient_mgmtneg.drop(mgmt_column, axis=1, inplace= True)
    df_patient_mgmtpos.drop(mgmt_column, axis=1, inplace= True)
    print("MGMT POS shape: {}, removed {} columns".format(df_patient_mgmtpos.shape, len(mgmt_column)))
    print("MGMT NEG shape: {}, removed {} columns".format(df_patient_mgmtneg.shape, len(mgmt_column)))
    return df_patient_mgmtpos, df_patient_mgmtneg

In [38]:
# All function Calls
import os
import cv2
import numpy as np
import pandas as pd

PATH = path + 'Data/BRATS/mod_data1/'
Work_Dir = PATH + 'Working_Data/'

workdir = os.listdir(Work_Dir)
if '.DS_Store' in workdir:
    workdir.remove('.DS_Store')

# Calling the fuctions casewise
for case in workdir:
    # os.system('say "{} started"'.format(case))
    Case_Path = os.path.join(Work_Dir, case)
    Split_Path =  Case_Path + '/train/'

    # Creating dataframe for individual patients
    df_patient_final_mgmtpos_new, df_patient_final_mgmtneg_new = Patient_Data(Split_Path,case, df_patient_final_mgmtpos, df_patient_final_mgmtneg)
    # df_patient_final_mgmtpos_new.to_csv('MGMT-POS-{}.csv'.format(case))
    # df_patient_final_mgmtneg_new.to_csv('MGMT-NEG-{}.csv'.format(case)) 
    # Eleminating rows after mad
    # df_patient_mgmtpos, df_patient_mgmtneg = MAD(df_patient_final_mgmtpos_new, df_patient_final_mgmtneg_new,case)
    df_patient_mgmtpos, df_patient_mgmtneg = df_patient_final_mgmtpos_new, df_patient_final_mgmtneg_new
    os.chdir(Split_Path)
    df_patient_mgmtpos.to_csv('MGMT-POS-MAD-{}.csv'.format(case))
    df_patient_mgmtneg.to_csv('MGMT-NEG-MAD-{}.csv'.format(case))
    
os.system('say "Feature Extraction is complete"')

Reading data for Case_5 : 


100%|██████████| 160/160 [06:21<00:00,  2.39s/it]


Reading data for Case_2 : 


100%|██████████| 160/160 [06:33<00:00,  2.46s/it]


Reading data for Case_3 : 


...

Reading data for Case_4 : 


100%|██████████| 160/160 [06:43<00:00,  2.52s/it]


Reading data for Case_1 : 


100%|██████████| 160/160 [06:14<00:00,  2.34s/it]


0